In [7]:
import os
import numpy as np

from astropy.io.fits import getdata
from astropy.wcs import wcs
from astropy.io import fits
from astropy import units as u
from astropy import constants as con
from astropy.coordinates import SkyCoord

import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib

import aplpy

matplotlib.use('PDF')


In [15]:
#### importing FITS image as a HDU and extracting certain header information in Python

intensitymap = '/home/tcc/DATA/GC_Dspecies/ancillary/GC.CI.mnt0.image.fits'
intensity_scale = 1.0
if_success = False
try:
  
  # importing FITS image to a HDU
  Ihdu   = fits.open(intensitymap)
  
  # editing the FITS image by multiplying a scaling factor
  Ihdu[0].data = Ihdu[0].data * intensity_scale
  
  if_success = True

except:
  print('Unable to read the intensity FITS image. Please double check the image file.')
  print(intensitymap)

if ( if_success == True ):
  # Reading FITS header
  try:
    naxis1 = Ihdu[0].header['naxis1']
    naxis2 = Ihdu[0].header['naxis2']
    crval1 = Ihdu[0].header['crval1']
    crpix1 = Ihdu[0].header['crpix1']
    cdelt1 = Ihdu[0].header['cdelt1']
    crval2 = Ihdu[0].header['crval2']
    crpix2 = Ihdu[0].header['crpix2']
    cdelt2 = Ihdu[0].header['cdelt2']
    hduwcs = wcs.WCS( Ihdu[0].header)
  except:
    print( 'Warning. No coordinate headers' )

  try:
    bmaj = Ihdu[0].header['bmaj']
    bmin = Ihdu[0].header['bmin']
    bpa  = Ihdu[0].header['bpa']
  except:
    print('Warning. No header for synthesized beam size')


In [16]:
print(Ihdu[0].header)

SIMPLE  =                    T /Standard FITS                                   BITPIX  =                  -32 /Floating point (32 bit)                         NAXIS   =                    4                                                  NAXIS1  =                 1536                                                  NAXIS2  =                 1536                                                  NAXIS3  =                    1                                                  NAXIS4  =                    1                                                  EXTEND  =                    T                                                  BSCALE  =   1.000000000000E+00 /PHYSICAL = PIXEL*BSCALE + BZERO                 BZERO   =   0.000000000000E+00                                                  BMAJ    =   1.753598782751E-04                                                  BMIN    =   1.189928501844E-04                                                  BPA     =   8.978679656982E+01          

In [17]:
### convert the central pixel in the image to the R.A.and Decl.values 
### and then convert them back at the x and y pixel numbers

xpix_temp = int(round( naxis1/2.0 ))
ypix_temp = int(round( naxis2/2.0 ))

world = hduwcs.wcs_pix2world( 
                             [ [xpix_temp, ypix_temp, 0, 0] ], 0
                            )

ra_center  = world[0][0]
dec_center = world[0][1]

pixcrd = hduwcs.wcs_world2pix([ [ra_center, dec_center, 0, 0] ], 0)
ra_center_pix  = pixcrd[0][0]
dec_center_pix = pixcrd[0][1]


In [18]:
####The value of a certain pixel in a (two dimensional image) is simply
xpix = 1030 - 1
ypix = 573 - 1
value = Ihdu[0].data[0][0][ypix][xpix]

print(value)

9.618184


In [20]:
####The (two-dimensional) image can be output as a plot using the APLpy commands

figsize = [9.0, 9.0]
cmap           = 'viridis'
plot_ticks     = True
tick_font      = 25
tick_ypad      = 0
plot_colorbar  = True
colorbar_location = 'top'
colorbar_width = 0.3
colorbar_pad = 0.3
colorbar_font = 15
colorbar_label = 'Colorbar'
colorbar_labelpad = 0.3
colorbar_labelfont = 20
width = 1.0
height = 1.0
plot_scalebar = True
distance      = 140.0
scalebar_size = 100.0
scalebar_text = '100 au'
scalebar_color = (0,0,0)
scalebar_font = 25.0
scalebar_linewidth = 3.0
plot_beam = True
beam_color = 'black'

if_plot = False
try:
    fig = aplpy.FITSFigure(Ihdu, figsize=(figsize[0], figsize[1]))
    if_plot = True

except:
    print('Unable to load or plot hdu. Please double check the image file.')

if ( if_plot == True ):
    vmax = np.nanmax(Ihdu[0].data)*1.2
    vmin = 0.0

    fig.show_colorscale(
                        vmax = vmax, vmin = vmin,
                        interpolation = 'bicubic',
                        cmap = cmap
                       )

# ticks
fig.tick_labels.set_font(size=tick_font)
fig.axis_labels.set_font(size=tick_font)
fig.axis_labels.set_ypad(tick_ypad)
if (plot_ticks != True):
    fig.axis_labels.hide_x()
    fig.axis_labels.hide_y()
    fig.tick_labels.hide_x()
    fig.tick_labels.hide_y()

# recentering
if ( (ra_center != 0) or (dec_center !=0) ):
    print('recentering')
    fig.recenter( [ra_center, dec_center, 0, 0], 0, 0,
                   width=width, height=height
                 )

# plot color bar
if (plot_colorbar == True):
    fig.add_colorbar()
    fig.colorbar.show()
    fig.colorbar.set_location(colorbar_location)
    fig.colorbar.set_width(colorbar_width)
    fig.colorbar.set_pad(colorbar_pad)
    fig.colorbar.set_font(size=colorbar_font, weight='medium', \
                               stretch='normal', family='sans-serif', \
                               style='normal', variant='normal')

    fig.colorbar.set_axis_label_text(colorbar_label)
    fig.colorbar.set_axis_label_font(size=colorbar_labelfont)
    fig.colorbar.set_axis_label_pad(colorbar_labelpad)
    
# plot scalebar
if (plot_scalebar == True):

    fig.add_scalebar( scalebar_size * (1.0/distance) / 3600.0)
    fig.scalebar.set_label(scalebar_text)
    fig.scalebar.set_color(scalebar_color)
    fig.scalebar.set_font(size=scalebar_font)
    fig.scalebar.set_linewidth(scalebar_linewidth)


INFO: Setting slices=[0, 0] [aplpy.core]
recentering


TypeError: WCS projection has 4 dimensions, so expected 2 (an Nx4 array and the origin argument) or 5 arguments (the position in each dimension, and the origin argument). Instead, 3 arguments were given.

In [ ]:
# to overplot another image as contours
fig.show_contour(chdu, colors    =ccolor, 
                       linestyles=clinestyle,
                       levels    =clevels,
                       linewidths=clinewidth)

#save an output figure
fig.save(outfigname, transparent=True)

In [ ]:
plt.show(data)